# Forecasting Road Speed with DiDi Data: STA 141B Final Project
### Group 28: Saurabh Maheshwari & Sarah Grajdura

In [ ]:
# import required libraries
import pandas as pd
import sqlite3
import sqlalchemy as sqla
from datetime import datetime
from dateutil import tz
import time
import eviltransform
import folium
import matplotlib.pyplot as plt
import geopy.distance
import numpy as np
import itertools
import pickle
import math
%matplotlib inline
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Conv1D, Dropout, MaxPooling1D, Flatten, Reshape, Activation, BatchNormalization, TimeDistributed, Bidirectional, Conv2D
from keras.backend import expand_dims
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam 
import tensorflow as tf
from numpy.random import seed
from tensorflow import set_random_seed
from sklearn.preprocessing import StandardScaler

In [ ]:
# Change unix time to china time
def unix_to_china(time_stamp, from_zone = tz.gettz('UTC'), to_zone = tz.gettz('Asia/Shanghai')):
    '''
    arguments: 
    time_stamp: unix time stamp
    from_zone, to_zone
    
    output:
    china time
    '''
    u = datetime.utcfromtimestamp(int(time_stamp))
    u = u.replace(tzinfo=from_zone)
    u = u.astimezone(to_zone)
    return({'month': u.month, 'day': u.day, 'hour': u.hour, 'minute': u.minute, 'second': u.second})

In [ ]:
# Change china time to unix time
def china_to_unix(month, day, hour, minute, year = 2016, second = 59, to_zone = tz.gettz('Asia/Shanghai')):
    '''
    arguments:
    china time - month, day, hour, minute, year, second
    to_zone: converting from which zone?
    
    output:
    unix time
    '''
    
    u = datetime.timestamp(datetime(year,month,day,hour,minute,second).replace(tzinfo = to_zone))
    return(u)

In [ ]:
# Transfer data to sql
def data_to_sql(filepath, tabname, conn, chunksize = 5000000):
    '''
    arguments:
    filepath: file destination (sql)
    tabname: table name
    conn: connection to the database
    chunksize: chunk size to be read from the data frame
    
    output:
    Data is read to sql database
    '''
    
    start = time.time()
    chunks = pd.read_csv(filepath, chunksize=chunksize, header=None, 
                     names = ['driver_id', 'trip_id', 'timestamp', 'longitude', 'latitude'])
    print('File: ', filepath)
    #print('working on chunk: ', 1)
    chunk = next(chunks)
    chunk.to_sql(tabname, conn, if_exists='replace', index = False)
    for i, chunk in enumerate(chunks):
        print('working on chunk: ', i+2)
        chunk.to_sql(tabname, conn, if_exists='append', index = False)
    print('time taken to read file: ', time.time()-start)

In [ ]:
# Map the data
def track_driver(df, transform = True):
    '''
    arguments:
    df: dataframe consisting of coordinates of driver 
    transform: whether to transfrom from gcj to wgs system of coordinates?
    
    output:
    folium map
    '''
    driver_map = folium.Map(location=[34.2324012260476, 108.94615156073496],
                            zoom_start = 15)
    if transform:
        circle_data = [eviltransform.gcj2wgs(row.latitude, row.longitude) for row in df.itertuples()]
    else: 
        circle_data = [[row.latitude, row.longitude] for row in df.itertuples()]
    for i in range(len(circle_data)):
        s = str('t:'+ str(df['timestamp'].iloc[i]) +'; lon:' + str(df['longitude'].iloc[i]))
        folium.CircleMarker(location = circle_data[i], radius = 3, tooltip=s).add_to(driver_map)
    return(driver_map)

In [ ]:
# Calculate average speed with respect to direction for a given time interval
def speed_calc_time(df):
    '''
    argument:
    df: data frame for which speeds are to be calculated
    
    output: 
    average speeds
    '''
    speed_n = []
    speed_s = []
    unique_trip_ids = df.trip_id.unique()
    for i in unique_trip_ids:
        trip_df = df[df['trip_id'] == i]
        trip_n = trip_df[(trip_df['longitude']>=108.9468) & (trip_df['longitude']<108.94723)].sort_values('timestamp')
        speed_n.append(trip_speed(trip_n))
        trip_s = trip_df[trip_df['longitude']<=108.9468].sort_values('timestamp')
        speed_s.append(trip_speed(trip_s))
    return(np.nanmean(speed_n), np.nanmean(speed_s))

In [ ]:
# Calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    '''
    arguments:
    lat1, lon1: coordinates of the first position
    lat2, lon2: coordinates of the second position
    
    output:
    distance between 2 points
    '''
    earth_radius = 6371*1000  # m
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = earth_radius * c
    return d

In [ ]:
# Calculate average speed given the df containing 2 sets of coordinates
def speed_calc(df):
    '''
    Given dataframe (only 2 rows), calculate the average speed
    '''
    delta_t = (df['timestamp'].iloc[1] - df['timestamp'].iloc[0])/3600
    lat_init, lon_init = df['latitude'].iloc[0], df['longitude'].iloc[0]
    lat_fin, lon_fin = df['latitude'].iloc[1], df['longitude'].iloc[1]
    dis = calculate_distance(lat_init, lon_init, lat_fin, lon_fin)/1000/delta_t
    return(dis)

In [ ]:
# Calculate average speed given the df containing bunch of sets of coordinates sorted by timestamp
def trip_speed(df):
    '''
    Calculate average speed given the df containing bunch of sets of coordinates sorted by timestamp
    '''
    l = []
    try:
        for i in np.arange(2, df.shape[0]+1):
            l.append(speed_calc(df[(i-2):i]))
    except:
        l = np.NaN
    #print(l)
    return(np.mean(l))

In [ ]:
# To be used for approach 1
def create_sets(df_list, window = 24, pred_step = 3, div = 80, jump = 1):
    '''
    arguments:
    df_list: time series data
    div: number used to scale the speeds
    jump: how much to slide the window
    window, pred_step: window length, prediction length
    
    output:
    predictors and output array
    '''
    X = []
    y = []
    i = 0
    while i < (len(df_list) - pred_step - window + 1):
        X.append(df_list[i:(i+window)])
        y.append(df_list[(i+window):(i+window+pred_step)])
        i += jump
    X = pd.DataFrame(X).values/div
    y = pd.DataFrame(y).values/div
    y = y.reshape(y.shape[0], y.shape[1], 1)
    return(X, y)

In [ ]:
# to be used for approach 1
def data_creation_pipeline(window = 48, pred_step = 1, div = 80, val = ['11_28', '11_29'], test = ['11_30']):    
    '''
    arguments:
    window, pred_step, div as already defined
    val: days for validation set
    test: days for test set
    
    output:
    training, validation and test sets
    '''
    ## Fill NAs with the column mean
    north_df = pd.DataFrame(north_bound)
    south_df = pd.DataFrame(south_bound)
    north_df.fillna(north_df.mean(), inplace = True)
    south_df.fillna(south_df.mean(), inplace = True)

    cols = []
    for i,j in itertools.product([10,11], np.arange(1,31)):
        cols.append(str(i)+'_'+str(j))

    north_df = north_df[cols]
    south_df = south_df[cols]

    north_list_train = north_df.drop(val+test, axis = 1).values.T.reshape((1,-1)).tolist()
    south_list_train = south_df.drop(val+test, axis = 1).values.T.reshape((1,-1)).tolist()
    north_list_val = north_df[val].values.T.reshape((1,-1)).tolist()
    south_list_val = south_df[val].values.T.reshape((1,-1)).tolist()
    north_list_test = north_df[test].values.T.reshape((1,-1)).tolist()
    south_list_test = south_df[test].values.T.reshape((1,-1)).tolist()

    north_X_train, north_y_train = create_sets(north_list_train[0], window = window, pred_step = pred_step, div = div)
    south_X_train, south_y_train = create_sets(south_list_train[0], window = window, pred_step = pred_step, div = div)
    north_X_val, north_y_val = create_sets(north_list_val[0], window = window, pred_step = pred_step, div = div, jump = pred_step)
    south_X_val, south_y_val = create_sets(south_list_val[0], window = window, pred_step = pred_step, div = div, jump = pred_step)
    north_X_test, north_y_test = create_sets(north_list_test[0], window = window, pred_step = pred_step, div = div, jump = pred_step)
    south_X_test, south_y_test = create_sets(south_list_test[0], window = window, pred_step = pred_step, div = div, jump = pred_step)

    X_train = np.concatenate([north_X_train, south_X_train])
    X_val = np.concatenate([north_X_val, south_X_val])
    X_test = np.concatenate([north_X_test, south_X_test])

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    y_train = np.concatenate([north_y_train, south_y_train])
    y_val = np.concatenate([north_y_val, south_y_val])
    y_test = np.concatenate([north_y_test, south_y_test])

    return(X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
# to be used for approach 2
def create_sets_2(df_list, window1, window2, pred_step = 3, div = 80, jump = 1):
    '''
    arguments:
    df_list: time series data
    div: number used to scale the speeds
    jump: how much to slide the window
    window1, window2, pred_step: window lengths, prediction length
    
    output:
    predictors and output array
    '''
    X = []
    y = []
    i = 0
    while i < (len(df_list) - pred_step - window1 - window2 + 1):
        X.append(df_list[(i):(i+window1)]+df_list[(i+window1+pred_step):(i+window1+pred_step+window2)])
        y.append(df_list[(i+window1):(i+window1+pred_step)])
        i += jump
    X = pd.DataFrame(X).values/div
    y = pd.DataFrame(y).values/div
    y = y.reshape(y.shape[0], y.shape[1], 1)
    return(X, y)

In [ ]:
# To be used for approach 2
def train_val_test(df1, df2, div, div_n, div_s, pred_step, val, test, window1, window2, flow = True):
  '''
  arguments:
  df1, df2: north and south speed or flow data sets
  div: scaling number for speeds
  div_n, div_s: scaling factor for north and south bound flows
  pred_step, window1, window2: prediction length, window lengths
  val, test: validation and test days
  flow: whether df1, df2 flow datasets?
  
  output:
  training, validation and test data sets
  '''
    north_list_train = df1.drop(val+test, axis = 1).values.T.reshape((1,-1)).tolist()
    south_list_train = df2.drop(val+test, axis = 1).values.T.reshape((1,-1)).tolist()
    north_list_val = df1[val].values.T.reshape((1,-1)).tolist()
    south_list_val = df2[val].values.T.reshape((1,-1)).tolist()
    north_list_test = df1[test].values.T.reshape((1,-1)).tolist()
    south_list_test = df2[test].values.T.reshape((1,-1)).tolist()

    if (flow):
        north_X_train, north_y_train = create_sets_2(north_list_train[0], window1, window2, pred_step = pred_step, div = div_n)
        south_X_train, south_y_train = create_sets_2(south_list_train[0], window1, window2, pred_step = pred_step, div = div_s)
        north_X_val, north_y_val = create_sets_2(north_list_val[0], window1, window2, pred_step = pred_step, 
                                                 div = div_n, jump = pred_step)
        south_X_val, south_y_val = create_sets_2(south_list_val[0], window1, window2, pred_step = pred_step, 
                                                 div = div_s, jump = pred_step)
        north_X_test, north_y_test = create_sets_2(north_list_test[0], window1, window2, pred_step = pred_step, 
                                                   div = div_n, jump = pred_step)
        south_X_test, south_y_test = create_sets_2(south_list_test[0], window1, window2, pred_step = pred_step, 
                                                   div = div_s, jump = pred_step)
    else:
        north_X_train, north_y_train = create_sets_2(north_list_train[0], window1, window2, pred_step = pred_step, div = div)
        south_X_train, south_y_train = create_sets_2(south_list_train[0], window1, window2, pred_step = pred_step, div = div)
        north_X_val, north_y_val = create_sets_2(north_list_val[0], window1, window2, pred_step = pred_step, 
                                                 div = div, jump = pred_step)
        south_X_val, south_y_val = create_sets_2(south_list_val[0], window1, window2, pred_step = pred_step, 
                                                 div = div, jump = pred_step)
        north_X_test, north_y_test = create_sets_2(north_list_test[0], window1, window2, pred_step = pred_step, 
                                                   div = div, jump = pred_step)
        south_X_test, south_y_test = create_sets_2(south_list_test[0], window1, window2, pred_step = pred_step, 
                                                   div = div, jump = pred_step)

    X_train_data = np.concatenate([north_X_train, south_X_train])
    X_val_data = np.concatenate([north_X_val, south_X_val])
    X_test_data = np.concatenate([north_X_test, south_X_test])

    X_train_data = X_train_data.reshape(X_train_data.shape[0], X_train_data.shape[1], 1)
    X_val_data = X_val_data.reshape(X_val_data.shape[0], X_val_data.shape[1], 1)
    X_test_data = X_test_data.reshape(X_test_data.shape[0], X_test_data.shape[1], 1)

    y_train = np.concatenate([north_y_train, south_y_train])
    y_val = np.concatenate([north_y_val, south_y_val])
    y_test = np.concatenate([north_y_test, south_y_test])

    return(X_train_data, X_val_data, X_test_data, y_train, y_val, y_test)

In [ ]:
# To be used for approach 2
def data_creation_pipeline_flow(window1 = 60, window2 = 40, pred_step = 60, div = 80, div_n = 63, div_s = 48, 
                                val = ['11_28', '11_29'], test = ['11_30']):
    '''
    This function basically calls train_val_test and stacks speed and flow features
    arguments:
    window1, window2, pred_step, div, div_n, div_s, val, test: all defined above
    
    output:
    stacked speed and flow features to be used for training, validation and testing
    '''
    
    north_df = pd.DataFrame(north_bound)
    south_df = pd.DataFrame(south_bound)
    north_df.fillna(north_df.mean(), inplace = True)
    south_df.fillna(south_df.mean(), inplace = True)

    north_df_flow = pd.DataFrame(north_bound_flow)
    south_df_flow = pd.DataFrame(south_bound_flow)

    cols = []
    for i,j in itertools.product([10,11], np.arange(1,31)):
        cols.append(str(i)+'_'+str(j))

    north_df = north_df[cols]
    south_df = south_df[cols]
    north_df_flow = north_df_flow[cols]
    south_df_flow = south_df_flow[cols]

    X_train_speed, X_val_speed, X_test_speed, y_train, y_val, y_test = train_val_test(north_df, south_df, div, div_n,
                                                                                      div_s, pred_step, val, test, 
                                                                                      window1, window2, flow = False)
    X_train_flow, X_val_flow, X_test_flow, _, _, _ = train_val_test(north_df_flow, south_df_flow, div, div_n, div_s, 
                                                                    pred_step, val, test, window1, window2, flow = True)

    X_train = np.dstack([X_train_speed, X_train_flow])
    X_val = np.dstack([X_val_speed, X_val_flow])
    X_test = np.dstack([X_test_speed, X_test_flow])

    return(X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def fetch_speed(month, day, cur, hour_init = [0], hour_final = [23]):
    '''
    fetch speeds from the sql database
    
    arguments:
    month, day of the date
    cur: cursor to the sql database
    hour_init, hour_final: initial and final hours of extracting the data
    
    output:
    north and south bound speeds
    '''
    
    speed_n = []
    speed_s = []
    if len(str(day)) == 1:
        name = str(month)+'0'+str(day)
    else: 
        name = str(month)+str(day)
    print(name)
    min_init, min_final = 0, 59
    sec_init, sec_final = 0, 59
    for hr_init, hr_final in zip(hour_init, hour_final):
        cur.execute('''select driver_id, trip_id, longitude, latitude, timestamp 
        from {} where timestamp between (?) and (?) and latitude between (?) and (?) 
        and longitude between (?) and (?)'''.format("'"+name+"'"), 
                    (china_to_unix(month, day, hr_init, min_init, second=sec_init), 
                     china_to_unix(month, day, hr_final, min_final, second=sec_final),
                     34.2324012260476, 34.23940384395769, 108.94615156073496, 108.94765571288106))
        df = pd.DataFrame(cur.fetchall(), columns = ['driver_id', 'trip_id', 'longitude', 'latitude', 'timestamp'])
        for cols in ['timestamp', 'latitude', 'longitude']:
            if df[cols].dtype == 'object':
                df[cols] = df[cols].astype('float')
        for h,m in itertools.product(np.arange(hr_init, hr_final+1), np.arange(4,60,5)):
            m_init, m_final = m-4, m
            t_init = china_to_unix(month, day, h, m_init, second=sec_init)
            t_final = china_to_unix(month, day, h, m_final, second=sec_final)
            df_mini = df[(df['timestamp']>=t_init) & (df['timestamp']=<t_final)]
            print(df_mini.shape)
            speeds = speed_calc_time(df_mini)
            speed_n.append(speeds[0])
            speed_s.append(speeds[1])
    return(speed_n, speed_s)

In [ ]:
def flow_calc_time(df):
    '''
    argument:
    data frame for which the flow is to be calculated
    
    output:
    north and south bound flows for the data frame
    '''
    trip_n = df[(df['longitude']>=108.9468)]
    flow_n = trip_n['driver_id'].nunique()
    trip_s = df[df['longitude']<108.9468]
    flow_s = trip_s['driver_id'].nunique()
    return(flow_n, flow_s)

In [ ]:
def fetch_flow(month, day, cur, hour_init = [0], hour_final = [23]):
    '''
    fetch flow data from the sql database
    
    arguments:
    month, day of the date
    cur: cursor to the sql database
    hour_init, hour_final: initial and final hours of extracting the data
    
    output:
    north and south bound flow
    '''
    
    
    flow_n = []
    flow_s = []
    if len(str(day)) == 1:
        name = str(month)+'0'+str(day)
    else: 
        name = str(month)+str(day)
    print(name)
    min_init, min_final = 0, 59
    sec_init, sec_final = 0, 59
    for hr_init, hr_final in zip(hour_init, hour_final):
        cur.execute('''select driver_id, longitude, latitude, timestamp 
        from {} where timestamp between (?) and (?) and latitude between (?) and (?) 
        and longitude between (?) and (?)'''.format("'"+name+"'"), 
                    (china_to_unix(month, day, hr_init, min_init, second=sec_init), 
                     china_to_unix(month, day, hr_final, min_final, second=sec_final),
                     34.2324012260476, 34.23940384395769, 108.94615156073496, 108.94765571288106))
        df = pd.DataFrame(cur.fetchall(), columns = ['driver_id', 'longitude', 'latitude', 'timestamp'])
        for cols in ['longitude', 'latitude', 'timestamp']:
            if df[cols].dtype == 'object':
                df[cols] = df[cols].astype('float')
        for h,m in itertools.product(np.arange(hr_init, hr_final+1), np.arange(4,60,5)):
            m_init, m_final = m-4, m
            t_init = china_to_unix(month, day, h, m_init, second=sec_init)
            t_final = china_to_unix(month, day, h, m_final, second=sec_final)
            df_mini = df[(df['timestamp']>=t_init) & (df['timestamp']<t_final)]
            flow = flow_calc_time(df_mini)
            flow_n.append(flow[0])
            flow_s.append(flow[1])
    return(flow_n, flow_s)

In [ ]:
# do predictions
def decode_sequence(input_sequence):
    '''
    Do predictions for the decoder module
    
    argument: 
    input sequence of the time series
    
    output:
    future predictions
    '''
    encoding_h, encoding_c = encoder_infer.predict(input_sequence)
    target_seq = np.zeros((1,1,1))
    target_seq[0,0,0] = input_sequence[:,-1,:]
    decoded_seq = np.zeros((1,pred_step,1))
    for i in range(pred_step):
        output, h, c = decoder_infer_model.predict([target_seq, encoding_h, encoding_c])
        decoded_seq[0, i, 0] = output
        encoding_h = h
        encoding_c = c
    return(decoded_seq) 

In [ ]:
def create_ex_feature(speed, flow, north = True):
    '''
    create speed and flow features for the extra day file
    
    arguments: 
    speed, flow - speed and flow data for the extra day file
    north - whether to create north bound features or not
    
    output:
    speed and flow features for extra day file
    '''
    
    # Time steps for window1 and window 2 for extra day file for north and south bound tracks
    if north:
        R11, L11, R12, L12, R21, L21, R22, L22 = 68, 8, 164, 128, 188, 128, 284, 248
    else:
        R11, L11, R12, L12, R21, L21, R22, L22 = 63, 3, 159, 123, 183, 123, 279, 243
    speed_ex_1 = pd.concat([speed[L11:R11], speed[L12:R12]]).values
    speed_ex_1 = speed_ex_1.reshape(1, speed_ex_1.shape[0])
    flow_ex_1 = np.array(flow[L11:R11] + flow[L12:R12])
    flow_ex_1 = flow_ex_1.reshape(1, flow_ex_1.shape[0])
    
    speed_ex_2 = pd.concat([speed[L21:R21], speed[L22:R22]]).values
    speed_ex_2 = speed_ex_2.reshape(1, speed_ex_2.shape[0])
    flow_ex_2 = np.array(flow[L21:R21] + flow[L22:R22])
    flow_ex_2 = flow_ex_2.reshape(1, flow_ex_2.shape[0])
    
    feat1 = np.dstack([speed_ex_1, flow_ex_1])
    feat2 = np.dstack([speed_ex_2, flow_ex_2])
    return(feat1, feat2)

In [ ]:
# extra day speeds
speed_north = pd.read_csv(r'G:\DiDi_sql\Predictions\Predictions_north.csv', header = 0, na_values='x')
speed_south = pd.read_csv(r'G:\DiDi_sql\Predictions\Predictions_south.csv', header = 0, na_values='x')

In [ ]:
sp_n = speed_north['speed']
sp_s = speed_south['speed']

In [ ]:
# dates for which data is to be stored in sql database
filepath = r'I:\DiDi Data\gps_2016'
dates = np.arange(1001,1031).tolist()
dates.extend(np.arange(1101, 1131).tolist())

In [ ]:
# connection to the sqlite database
conn = sqlite3.connect(r'G:\DiDi_sql\DiDi_30days.sqlite')
cur = conn.cursor()

In [ ]:
# Put data in sql database
for date in dates:
    path = filepath + str(date)
    tabname = str(date)
    data_to_sql(path, tabname, conn)

In [ ]:
# extract speeds
north_bound = {}
south_bound = {}
start = time.time()
for month, day in itertools.product([10,11], np.arange(1,31)):
    speeds = fetch_speed(month, day, cur)
    north_bound[str(month)+'_'+str(day)] = speeds[0]
    south_bound[str(month)+'_'+str(day)] = speeds[1]
print('time taken: ', time.time()-start)

In [ ]:
# save north bound speeds 
with open('north_bound.pickle', 'wb') as file:
    pickle.dump(north_bound, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# save south bound speeds
with open('south_bound.pickle', 'wb') as file:
    pickle.dump(south_bound, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# extract north and south bound flows for north and south directions
north_bound_flow = {}
south_bound_flow = {}
start = time.time()
for month, day in itertools.product([10,11], np.arange(1,31)):
    flow = fetch_flow(month, day, cur)
    north_bound_flow[str(month)+'_'+str(day)] = flow[0]
    south_bound_flow[str(month)+'_'+str(day)] = flow[1]
print('time taken: ', time.time()-start)

In [ ]:
# save north bound flows
with open('north_bound_flow.pickle', 'wb') as file:
    pickle.dump(north_bound_flow, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# save south bound flows
with open('south_bound_flow.pickle', 'wb') as file:
    pickle.dump(south_bound_flow, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# fetch flow information for extra day file
flow = fetch_flow(12, 1, cur)
north_flow_ex = flow[0]
south_flow_ex = flow[1]

In [ ]:
# Create features for the extra day
north1_feat, north2_feat = create_ex_feature(sp_n, north_flow_ex, north = True)
south1_feat, south2_feat = create_ex_feature(sp_s, south_flow_ex, north = False)

In [ ]:
# save extra day features
with open('extra_day_sp_flow.pickle', 'wb') as file:
    pickle.dump([north1_feat, north2_feat, south1_feat, south2_feat], file)

# Approach 1 - Create Encoder - Decoder Model

In [ ]:
# get the required data
with open('north_bound.pickle', 'rb') as file:
    north_bound = pickle.load(file)
with open('south_bound.pickle', 'rb') as file:
    south_bound = pickle.load(file)

In [ ]:
# Create training, validation and test sets
window = 72
pred_step = 2
X_train, X_val, X_test, y_train, y_val, y_test = data_creation_pipeline(window = window, pred_step=pred_step, div = 80)
# Generate input for encoder_decoder network training
encoder_input = X_train
decoder_output = y_train
decoder_input = np.zeros(decoder_output.shape)
decoder_input[:,1:,:] = y_train[:,:-1,:]
decoder_input[:,0,:] = X_train[:,-1,:]

In [ ]:
## Develop encoder decoder module

# 1D-CNN part
seed(1)
set_random_seed(2)
dilation_rates = [2**i for i in range(6)]
encoder_inp = Input((None, 1))
x = encoder_inp
for dilation_rate in dilation_rates:
    x = Conv1D(filters=128,
               kernel_size=3,
               padding='causal',
               dilation_rate=dilation_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
x = MaxPooling1D(pool_size = 2, strides = 2)(x)
x = TimeDistributed(Dense(128))(x)

# training encoder_decoder
# encoder part
cells = 256
x = LSTM(cells, return_sequences=True)(x)
x = Dropout(0.2)(x)
_, encoder_h, encoder_c = LSTM(cells, return_state = True)(x)
encoder_out = [encoder_h, encoder_c]

# decoder part 
decoder_inp = Input((None, 1))
decoder_lstm = LSTM(cells, return_sequences=True, return_state=True)
x, _, _ = decoder_lstm(decoder_inp, initial_state = encoder_out)
decoder_dense = Dense(1)
x = decoder_dense(x)
x_out = x

# train model
encoder_decoder_model = Model(inputs = [encoder_inp, decoder_inp], outputs = x_out)
adam = Adam(0.0001)
encoder_decoder_model.compile(adam, 'mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights=True)
batch_size = 128
n_epochs = 100
seed(124)
print('training encoder_decoder_model...')
encoder_decoder_model.fit(x = [encoder_input, decoder_input], y = decoder_output, validation_split=0.2, epochs=n_epochs, batch_size=batch_size, 
                         callbacks = [early_stopping])

# encoder_decoder_inference part
# get encodings
encoder_infer = Model(encoder_inp, encoder_out)
# decoder inference part
decoder_infer_h = Input((cells, ))
decoder_infer_c = Input((cells, ))
decoder_infer_input = [decoder_infer_h, decoder_infer_c]
# use same decoder LSTM and dense layers as used while training 
decoder_infer_output, state_h, state_c = decoder_lstm(decoder_inp, initial_state = decoder_infer_input) 
decoder_infer_states = [state_h, state_c]
decoder_infer_output = decoder_dense(decoder_infer_output)
decoder_infer_model = Model([decoder_inp] + decoder_infer_input, [decoder_infer_output]+decoder_infer_states)

In [ ]:
# Create predictions for validation set
pred_val = np.zeros(y_val.shape)
for i in range(X_val.shape[0]):
    pred_val[i,:,:] = decode_sequence(X_val[(i):(i+1),:,:])

In [ ]:
# Create inference for continuous 60 steps in the test set
test = pd.DataFrame(north_bound['11_30'])/80
test.fillna(test.mean(), inplace = True)
test = test.values
X_test_infer = test[:window, :]
X_test_infer_re = X_test_infer.reshape((1, window, 1))
pred_infer = decode_sequence(X_test_infer_re)*80
pred_infer_re = pred_infer.reshape(pred_step, 1)
for i in range(30):
    X_test_infer = np.concatenate([X_test_infer[pred_step:, :], pred_infer_re], axis = 0)
    X_test_infer_re = X_test_infer.reshape((1, window, 1))
    pred_infer_re = decode_sequence(X_test_infer_re)*80
    pred_infer = np.concatenate([pred_infer, pred_infer_re], axis = 0)
    pred_infer_re = pred_infer_re.reshape(pred_step, 1)

# Approach 2: Bi-Directional CNN LSTM

In [ ]:
# load the required data
with open('north_bound_flow.pickle', 'rb') as file:
    north_bound_flow = pickle.load(file)
with open('south_bound_flow.pickle', 'rb') as file:
    south_bound_flow = pickle.load(file)
with open('north_bound.pickle', 'rb') as file:
    north_bound = pickle.load(file)
with open('south_bound.pickle', 'rb') as file:
    south_bound = pickle.load(file)

In [ ]:
# Develop bi-directional CNN-LSTM model
seed(1)
set_random_seed(2)
# 1-D CNN part
dilation_rates = [2**i for i in range(6)]
encoder_inp = Input((window1+window2, 2))
x = encoder_inp
for dilation_rate in dilation_rates:
    x = Conv1D(filters=128,
               kernel_size=3,
               padding='same',
               dilation_rate=dilation_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
x = MaxPooling1D(pool_size = 2, strides = 2)(x)
x = TimeDistributed(Dense(128))(x)
x = Dropout(0.2)(x)
# Bi-Directional LSTM part
cells = 256
x = Bidirectional(LSTM(cells))(x)
x = Dropout(0.2)(x)
x = Dense(pred_step)(x)
x = Reshape((pred_step, 1))(x)
x_out = x
model = Model(inputs = [encoder_inp], outputs = [x_out])
model.summary()
# Model fitting
model.compile('adam', 'mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights=True)
batch_size = 128
n_epochs = 100
seed(124)
model.fit(X_train, y_train, validation_split=0.2, callbacks=[early_stopping], batch_size=batch_size, epochs = n_epochs)

In [ ]:
# get predictions on validation and test sets
pred_val = model.predict(X_val)
pred_test = model.predict(X_test)

# Generate predictions for extra day file

In [ ]:
# load extra day flow
with open('extra_day_sp_flow.pickle', 'rb') as file:
    extra_feat = pickle.load(file)
north1_feat, north2_feat, south1_feat, south2_feat = extra_feat

In [ ]:
# Generate predictions using CNN-LSTM bi-directional model
extra_n_1_pred = model.predict(north1_feat/80)*80
extra_n_2_pred = model.predict(north2_feat/80)*80
extra_s_1_pred = model.predict(south1_feat/80)*80
extra_s_2_pred = model.predict(south2_feat/80)*80

In [ ]:
# Fill the missing speeds in the north speed prediction file
sp_n_pred = sp_n.copy()
sp_n_pred[sp_n_pred.isnull()] = np.concatenate([extra_n_1_pred.flatten(), extra_n_2_pred.flatten()])

In [ ]:
# Fill the missing speeds in the south speed prediction file
sp_s_pred = sp_s.copy()
sp_s_pred[sp_s_pred.isnull()] = np.concatenate([extra_s_1_pred.flatten(), extra_s_2_pred.flatten()])

In [ ]:
# Save north speed prediction file
prediction_north = pd.Series(index=speed_north.iloc[:,0], data = sp_n_pred.values)
prediction_north.to_csv('prediction_north.csv')

In [ ]:
# Save south speed prediction file
prediction_south = pd.Series(index = speed_south.iloc[:,0], data = sp_s_pred.values)
prediction_south.to_csv('prediction_south.csv')